# Pozo Demo


### Step 1: Import Packages

In [ ]:
import lasio # Lasio reads well las files!
import pozo # Pozo visualizes them!

### Step 2: Import and Verify Data

In [ ]:
las = lasio.read("https://raw.githubusercontent.com/geopozo/pozo-demo/main/SALADIN.LAS")

pozo.units.check_las(las)

In [ ]:
graph1 = pozo.Graph(las, unit_map={"THOR":"ppm", "POTA":"fraction"})
graph1.render(height=600, override_theme={"color":"black", "track_width":90}, depth_position=1)

### Step 3: Customize The Graph

In [ ]:
include = ["CALI", "CGR", "LLS", "ILM", "ILD", "LLD", "NPHI", "RHOB", "GR"]
graph2 = pozo.Graph(las,  yaxis_name="DEPT:1", include=include) 

# Combining CGR CALI
graph2.combine_tracks("CGR", "CALI")

# Combining Conductivities
graph2.combine_tracks("LLD","ILD","ILM","LLS") 

# Combining NPHI RHOB
graph2.combine_tracks("RHOB", "NPHI")


graph2.set_theme("cangrejo")
graph2.get_theme().set_fallback("track_width", 150)

# This could have a better interface
graph2.depth_notes['Mardie Greensand'] = dict(range = 1105, color="#00FFFF")
graph2.depth_notes['Barrow Group'] = dict(range = 1115, color="green")

graph2.render(depth_position=0, show_depth=True, depth=[1080, 1180])

### Step 4: Make a crossplot

In [ ]:
graph2.CrossPlot(
    x = graph2.get_traces(pozo.HasLog("NPHI"))[0],
    y = graph2.get_traces(pozo.HasLog("RHOB"))[0],
    colors = ["depth"],#, graph2.get_traces(pozo.HasLog("GR"))[0]],
    xrange=(45, -15),
    yrange=(1.95, 2.945),
    size=800,
    depth_range=(1100, 1300),
).render()

### Step 5: Make an interactive crossplot

In [ ]:

graph2.render(height=800, depth_position=1, xp=graph2.xp, depth=[1100, 1300],)

In [ ]:


trace = graph2.get_traces()[0]
depth = trace.get_depth()
window = 25
start = 1100
end = 1200
start_index = trace.find_nearest(start)[0]
window_index = trace.find_nearest(start+window)[0] - start_index
end_index = trace.find_nearest(end)[0]

frame_count = range(start_index, end_index-window_index, 1)
paths = []
for i, cursor in enumerate(frame_count):
    print(f"{i} / {len(frame_count)}")
    graph2.depth_notes['Depth Highlight'] = dict(range = (depth[cursor], depth[cursor+window_index]), show_text=False)
    graph2.render(
        height=800,
        xp=graph2.xp,
        depth_position=1,
        color_lock={'depth': [start, end]},
        depth=[start, end],
        xp_depth_by_index=[cursor, cursor+window_index],
        static=True
    ).write_image("frames/"+str(i)+".png", engine="kaleido")
    paths.append("frames/"+str(i)+".png")
import imageio # pip install imageio[ffmpeg]
ims = [imageio.imread(f) for f in paths]
imageio.mimwrite("frames/all.mp4", ims, fps=3)


In [ ]:
# Works good to compare a whole range, no resolution (show zoomed in and zoomed out)

#graph2.last_fig.set_color_range("depth", (1100, 1800))

In [ ]:
#graph2.last_fig.add_shape(type="line", x0=.5, x1=.8, y0=1120, y1=1120, xref='paper', yref="y2", line_dash="dot")

#graph2.last_fig.add_shape(type="line", x0=100, x1=100, y0=0, y1=1, xref='x2', yref="paper", line_dash="dot")

# we can use paper and regular line if maybe we store a bit... in the figure? in the renderer? what are our coordinates
# we need it but we kinda need to see if it works